# 神经网络



加载matlab数据时可以使用scipy工具


将处理手写字符集没使用反向传播的前馈神经网络  
实现非正则化和正则化的版本，并且随即权重初始化。

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat

In [5]:
data = loadmat('./ex4data1.mat')
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [6]:
X=data['X']
y=data['y']

要对离散数据进行编码，也是新工具很实用，one-hot标签,可以变成神经网络输出层的标签

In [7]:
from sklearn.preprocessing import OneHotEncoder as oh

encoder=oh(sparse=False)
y_onehot = encoder.fit_transform(y)
y_onehot.shape


(5000, 10)

In [8]:
y[0], y_onehot[0,:]#可以看到10变为向量

(array([10], dtype=uint8), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]))

## sigmod 函数

In [24]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoid_gradient(z):#对sigmoid求导
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))

## 前向传播函数 
这里用的是三层神经网络  
 (400 + 1) -> (25 + 1) -> (10)

In [10]:
def forward(X,theta1,theta2):
    m=X.shape[0]
    
    a1 = np.insert(X, 0, values=np.ones(m), axis=1)#插入bias列
    z2=a1*theta1.T#这里的theta1是二阶矩阵
    a2 = np.insert(sigmoid(z2), 0, values=np.ones(m), axis=1)
    z3=a2*theta2.T
    final=sigmoid(z3)
    
    return a1,z2,a2,z3,final

代价公式：

$$
J(\theta)=\frac{1}{m} \sum_{i=1}^{m} \sum_{k=1}^{K}\left[-y_{k}^{(i)} \log \left(\left(h_{\theta}\left(x^{(i)}\right)\right)_{k}\right)-\left(1-y_{k}^{(i)}\right) \log \left(1-\left(h_{\theta}\left(x^{(i)}\right)\right)_{k}\right)\right]$$

In [11]:
def cost(params,input_size,hidden_size,num_labels,X,y,rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
       # reshape the parameter array into parameter matrices for each layer
    #就是给到的param是一维数组将其根据隐藏单元个数
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # 使用向前传播
    a1,z2,a2,z3,final = forward(X,theta1,theta2)
    
    #计算cost
    first=np.multiply(-y,np.log( final))
    second=np.multiply((1-y),np.log(1- final))
    
    return np.sum(first-second)/m
    
    
    
    

初始化设置

In [16]:
#初始化设置
input_size = 400#这是输入的特征数量
hidden_size = 25#隐藏单元的数量
num_labels = 10#输出的标签数量
learning_rate = 1#学习率

# 随机初始化完整网络参数大小的参数数组
params = (np.random.random(size=hidden_size * (input_size + 1) + num_labels * (hidden_size + 1)) - 0.5) * 0.25
m = X.shape[0]
X = np.matrix(X)
y = np.matrix(y)
                                                                               
#这里解开矩阵，但是不知道为什么函数里已经有的功能还要再实现一次                                                                               # 将参数数组解开为每个层的参数矩阵
theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

theta1.shape, theta2.shape
                                                                               
                                                                               


((25, 401), (10, 26))

In [15]:
params = (np.random.random(size=hidden_size * (input_size + 1) + num_labels * (hidden_size + 1)) - 0.5) * 0.25

params.shape

(10285,)

In [21]:
cost(params, input_size, hidden_size, num_labels, X, y_onehot, learning_rate)

7.284818017666802

## 正则化的cost公式

\begin{aligned}
J(\theta)= & \frac{1}{m} \sum_{i=1}^{m} \sum_{k=1}^{K}\left[-y_{k}^{(i)} \log \left(\left(h_{\theta}\left(x^{(i)}\right)\right)_{k}\right)-\left(1-y_{k}^{(i)}\right) \log \left(1-\left(h_{\theta}\left(x^{(i)}\right)\right)_{k}\right)\right]+ \\
& \frac{\lambda}{2 m}\left[\sum_{j=1}^{25} \sum_{k=1}^{400}\left(\Theta_{j, k}^{(1)}\right)^{2}+\sum_{j=1}^{10} \sum_{k=1}^{25}\left(\Theta_{j, k}^{(2)}\right)^{2}\right]
\end{aligned}  



In [22]:
def cost(params,input_size,hidden_size,num_labels,X,y,rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
       # reshape the parameter array into parameter matrices for each layer
    #就是给到的param是一维数组将其根据隐藏单元个数
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # 使用向前传播
    a1,z2,a2,z3,final = forward(X,theta1,theta2)
    
    #计算cost
    first=np.multiply(-y,np.log( final))
    second=np.multiply((1-y),np.log(1- final))
    
    J =np.sum(first-second)/m
    
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))
    #这边进行切片是因为第一个theta用于bias单元可以不用进行正则化
    
    return J
    
    
    

In [23]:
cost(params, input_size, hidden_size, num_labels, X, y_onehot, learning_rate)

7.290100040265456

## 反向传播算法
这是重点，用于更新计算减少训练数据上的网络误差  
算法的本质是链式计算出每一层的误差




In [28]:
def backprop(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward(X, theta1, theta2)
    
    # initializations
    J = 0
    delta1 = np.zeros(theta1.shape)  # (25, 401)
    delta2 = np.zeros(theta2.shape)  # (10, 26)
    
    # compute the cost
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J / m
    
    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))
    
    # perform backpropagation
    for t in range(m):
        a1t = a1[t,:]  # (1, 401)
        z2t = z2[t,:]  # (1, 25)
        a2t = a2[t,:]  # (1, 26)
        ht = h[t,:]  # (1, 10)
        yt = y[t,:]  # (1, 10)
        ## d3T,d2t是定义的变量可以和对应的a相乘变为对某个theta值的偏导数
        d3t = ht - yt  # (1, 10)
        
        z2t = np.insert(z2t, 0, values=np.ones(1))  # (1, 26)
        d2t = np.multiply((theta2.T * d3t.T).T, sigmoid_gradient(z2t))  # (1, 26)
        
        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + d3t.T * a2t
        
    delta1 = delta1 / m
    delta2 = delta2 / m
    
    # add the gradient regularization term
    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:] * learning_rate) / m
    delta2[:,1:] = delta2[:,1:] + (theta2[:,1:] * learning_rate) / m
    
    
    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
    
    return J, grad

In [30]:
J, grad = backprop(params, input_size, hidden_size, num_labels, X, y_onehot, learning_rate)
grad

array([-2.06800554e-03, -3.89117180e-06, -1.04485166e-05, ...,
        1.71866718e-01,  1.91232991e-01,  1.95154848e-01])